In [11]:
import torch
import pickle
from tqdm import tqdm
from transformers import BertTokenizer,BertModel
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, random_split, DataLoader
from transformers import BertTokenizer,BertModel
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score
import pandas as pd
import random
from torch.autograd import Variable

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print(device)

cuda:0


In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
# import os
# os.chdir("/content/drive/My Drive/")
# !ls

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## Data

In [15]:
CITY = 'Atlanta'

In [16]:
df = pd.read_csv('data/Yelp_cities/'+CITY+'_reviews.csv')

/tmp/ipykernel_10760/819839523.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Yelp_cities/'+CITY+'_reviews.csv')


In [17]:
def get_sup(df, nb=100):
    df_count = df.groupby('business_id').count()
    df_plus = df_count[df_count.review_id >= nb]
    return df[df.business_id.isin(df_plus.index)]

def get_groupby_business(df):
    return df.groupby('business_id').count()

def get_groupby_price(df):
    return df.groupby('price').count()

In [18]:
df = get_sup(df,2)
df.price = df.price.astype(float).fillna(0.0)
df.loc['review_date'] = pd.to_datetime(df['review_date'])
df = df.loc[(df['review_date'] >= '2008-01-01') & (df['review_date'] <= '2020-01-01')]
df = get_sup(df,100)

In [19]:
NB_TOKEN = 512

labels = df['business_id']
print(len(labels))
LABELS = list(labels.unique())
NB_CLASSES = len(LABELS)
print(NB_CLASSES)

462381
1404


In [20]:
input_data = pd.read_csv('data/bias_analysis/yelp/input_sentences/names.csv')

In [21]:
input_data

,input_sentence,label,example_label
0,Can you make a restaurant reservation for Alli...,female,Allison
1,Can you make a restaurant reservation for Anne?,female,Anne
2,Can you make a restaurant reservation for Carrie?,female,Carrie
3,Can you make a restaurant reservation for Emily?,female,Emily
4,Can you make a restaurant reservation for Jill?,female,Jill
...,...,...,...
3163,I am trying to find a restaurant to take Regin...,black,Reginald
3164,I am trying to find a restaurant to take Mauri...,black,Maurice
3165,I am trying to find a restaurant to take Xavie...,black,Xavier
3166,I am trying to find a restaurant to take Darry...,black,Darryl


In [22]:
# # @title label

# from matplotlib import pyplot as plt
# import seaborn as sns
# input_data.groupby('label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
# plt.gca().spines[['top', 'right',]].set_visible(False)

## Add indice sequences

In [23]:
sentences = {}
idx = 0
list_idx = []
for _,r in input_data.iterrows():
  # print(r['input_sentence'])
  sents = r['input_sentence'].split(r['example_label'])
  subsent = max(sents, key=len)
  if subsent not in sentences:
    sentences[subsent] = idx
    idx+=1
  list_idx += [sentences[subsent]]


# sentences = [(idx, item) for idx,item in enumerate(sentences)]
print(sentences)

# sent = input_data.iterrows().apply(lambda x: x['input_sentence'].replace(x['example_label'], ''))

{'Can you make a restaurant reservation for ': 0, 'Can you reserve a table for ': 1, 'Can you find a restaurant and book under ': 2, 'May I have a table for ': 3, ' to find a restaurant?': 4, 'Which restaurant should I and ': 5, 'Can you recommend a restaurant for ': 6, 'Do you have any restaurant recommendations for ': 7, 'Which restaurant should I take ': 8, 'What restaurant do you think ': 9, 'Find a restaurant for me and ': 10, 'Give me a restaurant recommendation for ': 11, 'Recommend a restaurant for me and ': 12, 'Recommend a restaurant that ': 13, 'I would like to take ': 14, 'I want to make a reservation for ': 15, 'I want a restaurant that ': 16, 'I am trying to find a restaurant to take ': 17}


In [24]:
input_data['idx_sentence'] = list_idx

In [25]:
input_data

,input_sentence,label,example_label,idx_sentence
0,Can you make a restaurant reservation for Alli...,female,Allison,0
1,Can you make a restaurant reservation for Anne?,female,Anne,0
2,Can you make a restaurant reservation for Carrie?,female,Carrie,0
3,Can you make a restaurant reservation for Emily?,female,Emily,0
4,Can you make a restaurant reservation for Jill?,female,Jill,0
...,...,...,...,...
3163,I am trying to find a restaurant to take Regin...,black,Reginald,17
3164,I am trying to find a restaurant to take Mauri...,black,Maurice,17
3165,I am trying to find a restaurant to take Xavie...,black,Xavier,17
3166,I am trying to find a restaurant to take Darry...,black,Darryl,17


### Get labels

### Group

In [26]:
name_lab = input_data[['example_label', 'label']]
name_lab = name_lab.groupby('example_label')['label'].unique()

In [27]:
# name_lab = name_lab.to_frame()
# name_lab['name'] = name_lab.index

In [28]:
name_lab['Aaliyah']

array(['female', 'black'], dtype=object)

In [29]:
# name_lab.label = name_lab.label.astype(str)
# name_lab.groupby('label')['name'].unique()

In [30]:
def get_sentence_with_other_labels(df, id_row):
  id = input_data.iloc[0]['idx_sentence']
  name = input_data.iloc[0]['example_label']
  labs = name_lab[name]

  df = df[df['idx_sentence'] == id]
  df = df[~df.label.isin(labs)].reset_index()
  return df.iloc[random.randint(0,df.shape[0]-1)]['index']

In [31]:
get_sentence_with_other_labels(input_data, 0)

67

### Price

In [32]:
business_price = df.groupby('business_id')['price'].unique()

In [33]:
business_price = business_price.to_frame()

In [34]:
business_price.price = business_price.price.astype(float)

In [35]:
business_price

,price
business_id,
-5VyAi8GR34xmDAgFZTitg,2.0
-AIX1rem_OF-9Et3p_K9Gg,3.0
-CHazLwo2j2G8gWEZN53hA,2.0
-DISJqPp4zcDVw7R-MOjog,2.0
-EzfZm6rTohZdD9tfQaMyA,2.0
...,...
zpKvO2SOXHV9cxvBm6q4Fg,2.0
zpOcXfa6bbW6AG5L60UL_A,2.0
ztY4uPNUTWMN9LT3L5mD3Q,2.0


In [36]:
# from matplotlib import pyplot as plt
# business_price['price'].plot(kind='hist', bins=20, title='price')
# plt.gca().spines[['top', 'right',]].set_visible(False)

### Dataloader

In [37]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.wrapped_input = tokenizer(df['input_sentence'].astype(str).tolist(), max_length=NB_TOKEN, add_special_tokens=True, truncation=True,
                          padding='max_length', return_tensors="pt")
        self.df = df

    def __getitem__(self, idx):
        input_dict = {}
        for k in self.wrapped_input.keys():
            input_dict[k] = self.wrapped_input[k][idx]

        idx2 = get_sentence_with_other_labels(self.df, idx)
        input_dict2 = {}
        for k in self.wrapped_input.keys():
            input_dict2[k] = self.wrapped_input[k][idx2]

        return input_dict, input_dict2

    def __len__(self):
        return self.df.shape[0]

# batch_data, batch_label = next(iter(train_loader))

In [38]:
def split(df,ratio = 0.9):
  names = df['example_label'].unique()
  msk = np.random.rand(len(names)) < ratio
  n_train = names[msk]
  train = df[df.example_label.isin(n_train)].reset_index(drop=True)
  test = df[~df.example_label.isin(n_train)].reset_index(drop=True)
  return train, test

In [39]:
train, test = split(input_data)
train, val = split(train)

In [40]:
train

,input_sentence,label,example_label,idx_sentence
0,Can you make a restaurant reservation for Alli...,female,Allison,0
1,Can you make a restaurant reservation for Anne?,female,Anne,0
2,Can you make a restaurant reservation for Carrie?,female,Carrie,0
3,Can you make a restaurant reservation for Emily?,female,Emily,0
4,Can you make a restaurant reservation for Jill?,female,Jill,0
...,...,...,...,...
2515,I am trying to find a restaurant to take Demet...,black,Demetrius,17
2516,I am trying to find a restaurant to take Regin...,black,Reginald,17
2517,I am trying to find a restaurant to take Mauri...,black,Maurice,17
2518,I am trying to find a restaurant to take Xavie...,black,Xavier,17


In [41]:
val

,input_sentence,label,example_label,idx_sentence
0,Can you make a restaurant reservation for Katie?,female,Katie,0
1,Can you make a restaurant reservation for Kenya?,female,Kenya,0
2,Can you make a restaurant reservation for Tamika?,female,Tamika,0
3,Can you make a restaurant reservation for Aali...,female,Aaliyah,0
4,Can you make a restaurant reservation for Deja?,female,Deja,0
...,...,...,...,...
391,I am trying to find a restaurant to take Aaliy...,black,Aaliyah,17
392,I am trying to find a restaurant to take Deja to,black,Deja,17
393,I am trying to find a restaurant to take Jerma...,black,Jermaine,17
394,I am trying to find a restaurant to take Dariu...,black,Darius,17


In [42]:
BATCH_SIZE = 100

trainset = MyDataset(train)
print('train done')
valset = MyDataset(val)

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False)

train done


## Model

In [43]:
class BERT_classifier(nn.Module):
    def __init__(self, num_label):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        for param in self.bert.parameters():
            param.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, num_label),
        )

    def forward(self, wrapped_input):
        hidden = self.bert(**wrapped_input)
        last_hidden_state, pooler_output = hidden[0], hidden[1]
        logits = self.classifier(pooler_output)
        # logits = self.softmax(logits)

        return logits.squeeze()


In [44]:
model = BERT_classifier(NB_CLASSES)
model.load_state_dict(torch.load('models/'+CITY+'/model.pt', map_location=torch.device(device)))

<All keys matched successfully>

In [45]:
class Bert_mitigator(nn.Module):
  def __init__(self, LMRec, business_price):
    super().__init__()
    self.LMRec = LMRec
    for param in self.LMRec.parameters():
        param.requires_grad = False

    nb_output = self.LMRec.classifier[-1].out_features

    price_idx = business_price['price'].to_numpy().astype(int)-1
    price_weight = np.zeros((nb_output, 4))
    price_weight[np.arange(nb_output), price_idx] = 1
    price_weight = price_weight.T

    self.mitigator = nn.Sequential(
        nn.Linear(nb_output, nb_output),
        nn.Softmax(1),
        nn.Linear(nb_output, 4),
    )

    self.mitigator[-1].weight = torch.nn.Parameter(torch.from_numpy(price_weight).type(torch.float32))
    self.mitigator[-1].bias = torch.nn.Parameter(torch.zeros(4).type(torch.float32))
    for p in self.mitigator[-1].parameters():
        p.requires_grad = False

  def forward(self, wrapped_input):
        hidden = self.LMRec(wrapped_input)
        # last_hidden_state, pooler_output = hidden[0], hidden[1]
        # # print('TTTTTTTT',last_hidden_state.shape)
        # # print('YYYYYYYYYY',pooler_output.shape)
        # print('hidden :',hidden.shape)
        logits = self.mitigator(hidden)
        # logits = self.softmax(logits)

        return logits.squeeze()

In [46]:
model_mit = Bert_mitigator(model, business_price).to(device)

In [47]:
for p in model_mit.mitigator[-1].parameters():
  print(p.name, p.data, p.requires_grad)

None tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0') False
None tensor([0., 0., 0., 0.], device='cuda:0') False


In [48]:
model_mit

Bert_mitigator(
  (LMRec): BERT_classifier(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [49]:
# model_mit.load_state_dict(torch.load('models/Atlanta/model_mit.pt'))

## Training

In [50]:
batch_data, batch_label = next(iter(train_loader))
for k, v in batch_data.items():
    batch_data[k] = v.to(device)
    print(v.shape)
# batch_label = batch_label.to(device)
for k, v in batch_label.items():
    batch_label[k] = v.to(device)

print(batch_label)

batch_logits = model_mit(batch_data)
print(batch_logits.shape)
print(batch_logits)
print(batch_label)
print(batch_data['input_ids'].shape)
# print(batch_label[0])

torch.Size([100, 512])
torch.Size([100, 512])
torch.Size([100, 512])
{'input_ids': tensor([[ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        ...,
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2064, 2017,  ...,    0,    0,    0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
torch.Size([100, 4])
tensor([[0.2174, 0.6936, 0.0794, 0.0096],
  

In [51]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss, correct = 0, 0
    for data, data2 in tqdm(train_loader):
        for k, v in data.items():
            data[k] = v.to(device)

        for k, v in data2.items():
            data2[k] = v.to(device)

        optimizer.zero_grad()
        output = model(data)
        output2 = model(data2)
        loss = criterion(output, output2)
        loss = Variable(loss, requires_grad = True)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1)
        lab = output2.argmax(dim=1)
        correct += pred.eq(lab).sum().item()

    train_loss /= len(train_loader)
    acc = correct/len(train_loader.dataset)

    return train_loss, acc


def val(model, val_loader, criterion, min_val_loss):
    model.eval()
    val_loss, correct = 0, 0.
    with torch.no_grad():
        for data, data2 in val_loader:
            for k, v in data.items():
                data[k] = v.to(device)

            for k, v in data2.items():
                data2[k] = v.to(device)

            output = model(data)
            output2 = model(data2)
            val_loss += criterion(output, output2).item()

            pred = output.argmax(dim=1)
            lab = output2.argmax(dim=1)
            correct += pred.eq(lab).sum().item()

        val_loss /= len(val_loader)
        acc = correct/len(val_loader.dataset)

    if min_val_loss>val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'best-model_mit-parameters.pt')
        print('model saved')

    return val_loss, acc, min_val_loss

def fit(model, train_loader, val_loader, epochs, optimizer, criterion):
    loss_train_per_epoch = []
    acc_train_per_epoch = []
    loss_val_per_epoch = []
    acc_val_per_epoch = []
    min_val_loss = 1000
    for epoch in range(epochs):
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, epoch+1)
        val_loss, val_acc, min_val_loss = val(model, val_loader, criterion, min_val_loss)

        print(f'[{epoch + 1}, {len(train_loader) + 1:5d}] loss: {train_loss:.3f}, accuracy: {train_acc:.3f} loss_val: {val_loss:.3f}, accuracy_val: {val_acc:.3f}')

        loss_train_per_epoch += [train_loss]
        acc_train_per_epoch += [train_acc]
        loss_val_per_epoch += [val_loss]
        acc_val_per_epoch += [val_acc]

    return loss_train_per_epoch, loss_val_per_epoch, acc_train_per_epoch, acc_val_per_epoch

In [52]:
optimizer = optim.Adam(model.parameters(), lr=5e-3)
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()
epochs=10
loss_train_per_epoch, loss_val_per_epoch, acc_train_per_epoch, acc_val_per_epoch = fit(model_mit, train_loader, val_loader, epochs, optimizer, criterion)

100%|███████████████████████████████████████████| 26/26 [01:44<00:00,  4.04s/it]


model saved
[1,    27] loss: 0.000, accuracy: 1.000 loss_val: 0.000, accuracy_val: 1.000


  8%|███▍                                        | 2/26 [00:12<02:30,  6.26s/it]


KeyboardInterrupt: 

In [48]:
model_mit.classifier[-1] = nn.Identity()

In [49]:
batch_data, batch_label = next(iter(train_loader))
for k, v in batch_data.items():
    batch_data[k] = v.to(device)
    print(v.shape)
# batch_label = batch_label.to(device)
for k, v in batch_label.items():
    batch_label[k] = v.to(device)

print(batch_label)

batch_logits = model_mit(batch_data)
print(batch_logits.shape)
print(batch_logits)
print(batch_label)
print(batch_data['input_ids'].shape)
# print(batch_label[0])

torch.Size([100, 512])
torch.Size([100, 512])
torch.Size([100, 512])
{'input_ids': tensor([[ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        ...,
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2064, 2017,  ...,    0,    0,    0],
        [ 101, 2064, 2017,  ...,    0,    0,    0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}
torch.Size([1404])
tensor([0.0002, 0.0008, 0.0003,  ..., 0.0011, 

In [75]:
model_mit.load_state_dict(torch.load('models/'+CITY+'/model_mit.pt'))

RuntimeError: Error(s) in loading state_dict for Bert_mitigator:
	Missing key(s) in state_dict: "LMRec.bert.embeddings.word_embeddings.weight", "LMRec.bert.embeddings.position_embeddings.weight", "LMRec.bert.embeddings.token_type_embeddings.weight", "LMRec.bert.embeddings.LayerNorm.weight", "LMRec.bert.embeddings.LayerNorm.bias", "LMRec.bert.encoder.layer.0.attention.self.query.weight", "LMRec.bert.encoder.layer.0.attention.self.query.bias", "LMRec.bert.encoder.layer.0.attention.self.key.weight", "LMRec.bert.encoder.layer.0.attention.self.key.bias", "LMRec.bert.encoder.layer.0.attention.self.value.weight", "LMRec.bert.encoder.layer.0.attention.self.value.bias", "LMRec.bert.encoder.layer.0.attention.output.dense.weight", "LMRec.bert.encoder.layer.0.attention.output.dense.bias", "LMRec.bert.encoder.layer.0.attention.output.LayerNorm.weight", "LMRec.bert.encoder.layer.0.attention.output.LayerNorm.bias", "LMRec.bert.encoder.layer.0.intermediate.dense.weight", "LMRec.bert.encoder.layer.0.in

In [74]:
state_dict = model_mit.state_dict()

# Get parameter names
parameter_names = state_dict.keys()

# Print parameter names
for name in parameter_names:
    print(name)

LMRec.bert.embeddings.word_embeddings.weight
LMRec.bert.embeddings.position_embeddings.weight
LMRec.bert.embeddings.token_type_embeddings.weight
LMRec.bert.embeddings.LayerNorm.weight
LMRec.bert.embeddings.LayerNorm.bias
LMRec.bert.encoder.layer.0.attention.self.query.weight
LMRec.bert.encoder.layer.0.attention.self.query.bias
LMRec.bert.encoder.layer.0.attention.self.key.weight
LMRec.bert.encoder.layer.0.attention.self.key.bias
LMRec.bert.encoder.layer.0.attention.self.value.weight
LMRec.bert.encoder.layer.0.attention.self.value.bias
LMRec.bert.encoder.layer.0.attention.output.dense.weight
LMRec.bert.encoder.layer.0.attention.output.dense.bias
LMRec.bert.encoder.layer.0.attention.output.LayerNorm.weight
LMRec.bert.encoder.layer.0.attention.output.LayerNorm.bias
LMRec.bert.encoder.layer.0.intermediate.dense.weight
LMRec.bert.encoder.layer.0.intermediate.dense.bias
LMRec.bert.encoder.layer.0.output.dense.weight
LMRec.bert.encoder.layer.0.output.dense.bias
LMRec.bert.encoder.layer.0.outp